In [1]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
import os

# Load API key
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

# Model client
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat


planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

In [3]:
from dotenv import load_dotenv
from langchain_community.utilities import GoogleSerperAPIWrapper
from autogen_ext.tools.http import HttpTool

serper_key = os.getenv("SERPER_API_KEY")

# Validate
if not serper_key:
    raise ValueError("Please set the SERPER_API_KEY environment variable.")

# Use the keys
os.environ['SERPER_API_KEY'] = serper_key

search_tool_wrapper = GoogleSerperAPIWrapper(type='search')

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found."   

In [4]:
def search_web_tool(query:str)-> str:
    # Simulate a web search
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."

In [5]:
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [6]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web_tool],
    reflect_on_tool_use=False,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do calculations or data analysis on them.
    ''',
)

In [7]:
def percentage_change_tool(start:float, end:float) -> float:
    # Calculate percentage change
    if start == 0:
        return 0
    return ((end - start) / start) * 100

In [8]:
data_analyst_agent = AssistantAgent(
    name = 'DataAnalystAgent',
    description= 'An agent for performing calculations and data analysis.',
    model_client=model_client,
    tools= [percentage_change_tool],
    system_message='''
        You are a data analyst agent.
        Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided (percentage_change_tool).

        If you have not seen the data, ask for it.

    ''',
)

### Termination Condition

In [9]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

text_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=20)
combined_termination = text_mention_termination | max_message_termination

In [10]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [11]:
planning_agent.description

'An agent for planning tasks, this agent should be the first to engage when given a new task.'

In [12]:
selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [13]:
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"


In [14]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To find the player with the highest points in the Miami Heat for the 2006-2007 season and then calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to carry out the following steps:

1. WebSearchAgent: Identify the Miami Heat player with the highest total points during the 2006-2007 NBA season.
2. WebSearchAgent: Gather total rebound statistics for this player for the 2007-2008 season.
3. WebSearchAgent: Gather total rebound statistics for the same player for the 2008-2009 season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds from the 2007-2008 to the 2008-2009 season for this player.

Let's start by delegating these tasks: 

1. WebSearchAgen

TaskResult(messages=[TextMessage(id='d03f8c8e-e4a5-46e4-8bae-bafdb245508a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 16, 3, 409592, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='c2111dc7-d320-4668-9815-e3ea7361b0de', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=250), metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 16, 9, 50827, tzinfo=datetime.timezone.utc), content="To find the player with the highest points in the Miami Heat for the 2006-2007 season and then calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to carry out the following steps:\n\n1. WebSearchAgent: Identify the Miami Heat player with the highest total 

In [15]:
# With real web search


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To determine the Miami Heat player with the highest points in the 2006-2007 season and calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to perform the following actions:

1. Identify the Miami Heat player with the highest total points during the 2006-2007 NBA season.
2. Collect the total rebounds data for this player in the 2007-2008 season.
3. Collect the total rebounds data for this player in the 2008-2009 season.
4. Calculate the percentage change in total rebounds between these two seasons.

Since the previous task data was identical, I will now summarize given the past details:

The Miami Heat player with the highest total points in the 2006-2007 season was 

TaskResult(messages=[TextMessage(id='d297a631-d8b1-4e8b-9fdc-38619b743e6b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 16, 21, 279874, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='d0acbe0d-853a-4225-8b9f-4b21b25ff9e1', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=757, completion_tokens=227), metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 16, 25, 485441, tzinfo=datetime.timezone.utc), content='To determine the Miami Heat player with the highest points in the 2006-2007 season and calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to perform the following actions:\n\n1. Identify the Miami Heat player with the highest total points during the 200

In [16]:
state = await selector_team.save_state()

In [17]:
state


{'type': 'TeamState',
 'version': '1.0.0',
 'agent_states': {'PlanningAgent': {'type': 'ChatAgentContainerState',
   'version': '1.0.0',
   'agent_state': {'type': 'AssistantAgentState',
    'version': '1.0.0',
    'llm_context': {'messages': [{'content': 'Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?',
       'source': 'user',
       'type': 'UserMessage'},
      {'content': "To find the player with the highest points in the Miami Heat for the 2006-2007 season and then calculate the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons, we need to carry out the following steps:\n\n1. WebSearchAgent: Identify the Miami Heat player with the highest total points during the 2006-2007 NBA season.\n2. WebSearchAgent: Gather total rebound statistics for this player for the 2007-2008 season.\n3. WebSearchAgent: Gather total rebound

### Custom Selector Function

In [18]:
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source == web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_selector_fun)

In [19]:
await selector_team.reset()

In [20]:
# With real web search
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address this complex task, we need to:

1. Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. Gather data on this player’s total rebounds for the 2007-2008 and 2008-2009 seasons.
3. Calculate the percentage change in total rebounds between these two seasons.

Let's break it down:

1. WebSearchAgent: Search for the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find the total rebounds for this player during the 2007-2008 NBA season.
3. WebSearchAgent: Find the total rebounds for the same player during the 2008-2009 NBA season.
4. DataAnalystAgent: Calculate the percentage change in total rebounds between the 2007-2008 and 2008-2009 seasons for 

TaskResult(messages=[TextMessage(id='585acd4c-9f6c-4b2e-854c-d8695d1b13ed', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 16, 25, 601576, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='2feb5c72-9a80-43b9-83c8-6c34b08ab9e2', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=198), metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 16, 29, 171274, tzinfo=datetime.timezone.utc), content="To address this complex task, we need to:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. Gather data on this player’s total rebounds for the 2007-2008 and 2008-2009 seasons.\n3. Calculate the percentage change in total rebounds between these two seasons.\n\nLet's br

In [21]:
await selector_team.reset()


In [22]:
# If Data analyst agent is not selected after web search agent
task = "Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from typing import Sequence

def my_selector_fun(messages: Sequence[BaseAgentEvent | BaseChatMessage]):

    if messages[-1].source != web_search_agent.name:
        return data_analyst_agent.name
    return None


selector_team = SelectorGroupChat(
    participants=[planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=my_selector_fun)


from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat player's points in the 2006-2007 season and his total rebounds in the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat player's points in the 2006-2007 season and his total rebounds in the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat player's points in the 2006-2007 season and his total rebounds in the 2007-2008 and 2008-2009 seasons.
---------- TextMessage (DataAnalystAgent) ----------
Please provide the data for the Miami Heat player's points in the 2006-2007 season and his total rebounds in the 2007-2008 and 20

TaskResult(messages=[TextMessage(id='224d02f1-5819-4e92-b4e4-1f88193b2f9c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 16, 38, 718995, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='537f497e-3fca-45c4-b76b-6bc1ca64b980', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens=150, completion_tokens=41), metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 16, 40, 198886, tzinfo=datetime.timezone.utc), content="Please provide the data for the Miami Heat player's points in the 2006-2007 season and his total rebounds in the 2007-2008 and 2008-2009 seasons.", type='TextMessage'), TextMessage(id='5a1cd806-485b-4dd6-8ee6-4c8e3778c204', source='DataAnalystAgent', models_usage=RequestUsage(prompt_tokens=194, completion_

In [23]:
await selector_team.reset()

### Candidate Function

In [28]:
from typing import List, Sequence

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=10)
termination = text_mention_termination | max_messages_termination

def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]

In [29]:
# Reset the previous team and run the chat again with the selector function.
#await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- TextMessage (PlanningAgent) ----------
To find the answer to your query, we need to follow these steps:

1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.
2. WebSearchAgent: Retrieve the total rebounds statistics for this player for the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent: Calculate the percentage change in the player's total rebounds from the 2007-2008 season to the 2008-2009 season.

Let's proceed with these tasks now.
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_fV1nBmLeTNj618IChdWThDa3', arguments='{"query":"Miami Heat player highest points 2006-2007 season"}', name='search_web_tool')]
---------- ToolCallExecutionEvent (WebSearchAgent) ----------
[FunctionExecutionResult(content='Here are the total points scored by Miami Heat players in the 2006-2007 season:\n        Udonis Haslem: 844 points\n        Dwayne Wade: 1397 points\n        James Posey: 550 points\n        ...\n

TaskResult(messages=[TextMessage(id='dfc9ad58-b620-4fd0-9dc8-099e0a62bd3f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 24, 5, 507665, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='adf826f0-46d8-423f-877f-d5f4fd5e0001', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=113), metadata={}, created_at=datetime.datetime(2025, 7, 19, 12, 24, 8, 224473, tzinfo=datetime.timezone.utc), content="To find the answer to your query, we need to follow these steps:\n\n1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season.\n2. WebSearchAgent: Retrieve the total rebounds statistics for this player for the 2007-2008 and 2008-2009 seasons.\n3. DataAnalystAgent: Calculate 

In [ ]:
from autogen_agentchat.agents import UserProxyAgent

user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")

text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=10)
termination = text_mention_termination | max_messages_termination


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    

    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    

    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
        

    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
# await sele.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- TextMessage (PlanningAgent) ----------
To address your query, we need to perform the following tasks:

1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.
2. WebSearchAgent: Find total rebounds for this player for the 2007-2008 and 2008-2009 seasons.
3. DataAnalystAgent: Calculate the percentage change in total rebounds for the identified player between the 2007-2008 and 2008-2009 seasons.

I will start by delegating these tasks.

1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.

---------- TextMessage (UserProxyAgent) ----------
APPROVE
---------- ToolCallRequestEvent (WebSearchAgent) ----------
[FunctionCall(id='call_fG9DOWyvNIuPgauTfMuUQe0s', arguments

TaskResult(messages=[TextMessage(id='6e8c50dc-9448-4a79-a61d-60e6f81d960a', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 36, 48, 608056, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest point in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(id='c52e8378-2293-4d78-bbbb-f64081c534fe', source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=138), metadata={}, created_at=datetime.datetime(2025, 7, 19, 4, 36, 50, 878076, tzinfo=datetime.timezone.utc), content='To address your query, we need to perform the following tasks:\n\n1. WebSearchAgent: Identify the Miami Heat player with the highest points in the 2006-2007 NBA season.\n2. WebSearchAgent: Find total rebounds for this player for the 2007-2008 and 2008-2009 seasons.\n3. DataAnalystAgent: Calculate the percentag